<font size="8"> **Adding environmental data from available observations to unique background points** </font>  
In this notebook, we will extract environmental data from sea ice and sea surface temperature observations using the coordinates of the reported crabeater sightings.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/37059/status,
Dashboard: /proxy/37059/status,Workers: 7
Total threads: 14,Total memory: 56.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44613,Workers: 7
Dashboard: /proxy/37059/status,Total threads: 14
Started: Just now,Total memory: 56.00 GiB
Comm: tcp://127.0.0.1:34295,Total threads: 2
Dashboard: /proxy/39903/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:35743,


# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [4]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Biological_Data/BG_points/unique_background_20x_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,date,year,sector,longitude,latitude,xt_ocean,yt_ocean,zone,month,season_year,life_stage,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1987-11,1987,Central Indian,71.45,-69.65,71.45,-69.662,Antarctic,11,autumn,weaning,1980,0,NaN,NaN,NaN,NaN
1,1987-11,1987,Central Indian,73.05,-69.65,73.05,-69.662,Antarctic,11,autumn,weaning,1980,0,NaN,NaN,NaN,NaN
2,1996-11,1996,Central Indian,74.45,-69.65,74.45,-69.662,Antarctic,11,autumn,weaning,1990,0,89.985,-550.881,187.224,518.817
3,1998-11,1998,Central Indian,76.55,-69.55,76.55,-69.535,Subantarctic,11,autumn,weaning,1990,0,NaN,NaN,NaN,NaN
4,1996-11,1996,Central Indian,73.75,-69.45,73.75,-69.451,Antarctic,11,autumn,weaning,1990,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30666,1989-11,1989,Central Indian,74.05,-59.45,74.05,-59.442,Antarctic,11,autumn,weaning,1980,0,89.919,543.100,725.785,1618.292
30667,1998-11,1998,Central Indian,71.75,-59.25,71.75,-59.238,Antarctic,11,autumn,weaning,1990,0,89.976,559.524,720.119,4481.736
30668,1996-11,1996,Central Indian,76.35,-59.25,76.35,-59.238,Antarctic,11,autumn,weaning,1990,0,89.961,506.485,705.324,1267.889
30669,1989-11,1989,Central Indian,73.95,-58.85,73.95,-58.827,Subantarctic,11,autumn,weaning,1980,0,89.971,610.227,790.155,2223.069


## Loading environmental data from observations

In [14]:
#Creating dictionary with useful information
varDict = {'var_original': 'dist_ice',
           #Folder containing obs
           'obs_main': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs/Distance_Edge/north_south/*.nc',
           #Name to store variable in final data frame
           'var_short': 'dist_ice_edge_km',
           #Output folder
           'base_out': '../Environmental_Data/Env_obs'}

In [15]:
#Getting list of all obs in folder
files_var = sorted(glob(varDict['obs_main']))
var = varDict['var_original']

#Loading all data into single dataset
var_df = xr.open_mfdataset(files_var)[var]
#Renaming data array variable
var_df.name = varDict['var_short']
#Changing coordinates names if needed
if 'xt_ocean' not in var_df.coords:
    var_df = var_df.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})
#Selecting dates between 1981 and 2013 and for the Indian sectors
var_df = var_df.sel(time = slice('1981-11', '2013-12'), xt_ocean = slice(30, 170))
#Rechunking dataset
var_df = var_df.chunk((1, 135, 180))

#Checking results
var_df

<xarray.DataArray 'dist_ice_edge_km' (time: 386, yt_ocean: 740, xt_ocean: 1400)>
dask.array<rechunk-merge, shape=(386, 740, 1400), dtype=float64, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 30.05 30.15 30.25 30.35 ... 169.8 169.9 170.0
  * time      (time) datetime64[ns] 1981-11-15 1981-12-15 ... 2013-12-15

## Extracting environmental data
We will use the `latitude` and `longitude` columns together with the `event_date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [18]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

## Extracting data

In [19]:
#Extracting data
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short']).columns.tolist()

#Checking results
print(cols); var_pd

['yt_ocean', 'xt_ocean', 'year', 'month']


,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
dim_0,,,,,
313,-67.465,75.15,-442.943011,1981,12
20658,-64.331,75.35,-104.973925,1981,12
7426,-65.269,77.15,-186.808554,1981,12
2782,-66.240,77.65,-284.395662,1981,12
9918,-65.058,77.65,-156.536614,1981,12
...,...,...,...,...,...
17295,-64.633,146.95,-82.078142,2013,12
11107,-65.058,149.05,-158.077744,2013,12
20526,-64.461,149.95,-115.941450,2013,12


## Joining masked data frame with background data frame
We will use the grid cell coordinates and dates to perform this join.

In [20]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters = crabeaters.drop_duplicates()
crabeaters

,date,year,sector,longitude,latitude,xt_ocean,yt_ocean,zone,month,season_year,...,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,SIC,SST_degC,lt_pack_ice,dist_ice_edge_km
0,1987-11,1987,Central Indian,71.45,-69.65,71.45,-69.662,Antarctic,11,autumn,...,1980,0,NaN,NaN,NaN,NaN,NaN,-1.410222,0.000000,NaN
1,1987-11,1987,Central Indian,73.05,-69.65,73.05,-69.662,Antarctic,11,autumn,...,1980,0,NaN,NaN,NaN,NaN,NaN,-1.548782,0.000000,NaN
2,1996-11,1996,Central Indian,74.45,-69.65,74.45,-69.662,Antarctic,11,autumn,...,1990,0,89.985,-550.881,187.224,518.817,NaN,NaN,0.000000,NaN
3,1998-11,1998,Central Indian,76.55,-69.55,76.55,-69.535,Subantarctic,11,autumn,...,1990,0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
4,1996-11,1996,Central Indian,73.75,-69.45,73.75,-69.451,Antarctic,11,autumn,...,1990,0,NaN,NaN,NaN,NaN,NaN,-1.581244,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37880,1989-11,1989,Central Indian,74.05,-59.45,74.05,-59.442,Antarctic,11,autumn,...,1980,0,89.919,543.100,725.785,1618.292,0.153513,-1.342190,0.047619,-48.185653
37881,1998-11,1998,Central Indian,71.75,-59.25,71.75,-59.238,Antarctic,11,autumn,...,1990,0,89.976,559.524,720.119,4481.736,0.030800,-1.371573,0.035714,37.979983
37882,1996-11,1996,Central Indian,76.35,-59.25,76.35,-59.238,Antarctic,11,autumn,...,1990,0,89.961,506.485,705.324,1267.889,0.078127,-1.264371,0.071429,8.036973
37883,1989-11,1989,Central Indian,73.95,-58.85,73.95,-58.827,Subantarctic,11,autumn,...,1980,0,89.971,610.227,790.155,2223.069,0.095507,-1.304765,0.000000,5.751460


## Saving data frame to disk

In [21]:
#Ensure output folder exists
os.makedirs(varDict['base_out'], exist_ok = True)

#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_background_20x_obs_all_env.csv')

#Saving as csv file
crabeaters.to_csv(file_out, index = False)